In [ ]:
import json
from itertools import cycle
from collections import defaultdict
import uuid
import numpy as np
from vizbuilder import Html, components, require

In [ ]:
colors_iter = cycle(["red", "blue", "green", "orange", "purple"])
colors = defaultdict(lambda: next(colors_iter))

rand = np.random.RandomState(42)
data = [
        {"category": k, "subcategory": j, "start": int(i), "duration": rand.choice(20), "color": colors[k]}
        for k in ["A", "B", "C", "D", "E"] for j in range(3) for i in np.cumsum(rand.choice(20, 20) + 20) 
    ]

In [ ]:
# Original Gantt Chart (without scrubber for comparison)
container_id = f"d3-container-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(18)                    // Larger axis labels
        .titleFontSize(22)                   // Larger title
        .title(null)        // Custom title
        .xAxisLabel("Cycles")                  // Custom x label
        .yAxisLabel(null)                    // Hide y label
        .showGridLines(false)                // Clean look
        .categoryBackgroundOpacity([0.5, 0.1]) // Bolder groups
        .interactiveMode(false)              // Static export
        .colorScheme(  ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]); // Custom colors
          
    //.scrubberEnabled(false)
    //;  // Disable scrubber for this demo
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Basic Gantt Chart (no scrubber)"),
    Html.div(id_=container_id),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

In [ ]:
# Enhanced Gantt Chart with Interactive Scrubber and Linked Data Display
chart_container_id = f"chart-container-{uuid.uuid4().hex[:8]}"
display_container_id = f"display-container-{uuid.uuid4().hex[:8]}"

user_code = f"""
    //console.log("Starting to load modules...");
    
    const data={json.dumps(data)};
    
    try {{
        // Test if modules are available
        console.log("d3 available:", typeof d3);
        console.log("ganttChart available:", typeof ganttChart);
        console.log("dataDisplay available:", typeof dataDisplay);
        
        // Create the linked data display component first
        const displayContainer = d3.select("#{display_container_id}");
        const displayComponent = dataDisplay().title("Timeline Entries at Scrubber Position");
        displayContainer.call(displayComponent);
        //console.log("Data display component created successfully");
        
        // Create the Gantt chart with scrubber enabled
        const chartContainer = d3.select("#{chart_container_id}").datum(data);
        const chart = ganttChart().scrubberEnabled(true);
        
        // Link the components using the event system BEFORE rendering chart
        chart.on("scrubberMove", function(event) {{
            // console.log("Scrubber moved, intersected data:", event.intersectedData);
            // Update the data display with intersected entries
            const displayNode = displayContainer.node();
            displayNode.updateData(event.intersectedData);
        }});
        
        // Now render the chart (this will emit the initial scrubberMove event)
        chartContainer.call(chart);
        //console.log("Gantt chart created successfully");
        
        // Handle window resize for chart
        window.addEventListener('resize', (() => chartContainer.call(chart)));
        
        //console.log("All components initialized successfully");
        
    }} catch (error) {{
        console.error("Error in chart initialization:", error);
        console.error("Error stack:", error.stack);
    }}
"""

html = Html.body(
    Html.h3("Interactive Gantt Chart with Scrubber"),
    Html.p("Drag the orange scrubber line to see which timeline entries intersect. The data display below updates automatically."),
    Html.div(id_=chart_container_id, style="height: 350px; margin-bottom: 20px;"),
    Html.div(id_=display_container_id, style="height: 300px;"),
    Html.script(require(modules=["d3", "ganttChart", "dataDisplay"], user_code=user_code, clear_cache=True))
)
display(html)

In [ ]:
# Gantt Chart with Excalidraw Virgil Font (WORKS - No CORS issues)
# This is the only Excalidraw font publicly accessible without CORS restrictions

container_id = f"d3-container-virgil-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(14)
        .titleFontSize(24)
        .title("Project Timeline")
        .xAxisLabel("Time (cycles)")
        .yAxisLabel("Teams")
        .showGridLines(true)
        .categoryBackgroundOpacity([0.3, 0.05])
        .interactiveMode(true)
        .scrubberEnabled(true)
        .colorScheme(["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"])
        // Configure Excalidraw's Virgil font (publicly accessible)
        .titleFontFamily("Virgil, Segoe UI Emoji")
        .axisFontFamily("Virgil, Segoe UI Emoji") 
        .fontUrl("https://virgil.excalidraw.com/Virgil.woff2");
          
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Gantt Chart with Excalidraw Virgil Font"),
    Html.p("Virgil is Excalidraw's original handwritten font and works without CORS issues. Perfect for matching Excalidraw diagrams!"),
    Html.div(id_=container_id, style="height: 400px;"),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

In [ ]:
# Gantt Chart with Google Fonts - Indie Flower (Handwritten Alternative)
# Since Excalifont and Comic Shanns are CORS-blocked, use Google Fonts instead

container_id = f"d3-container-indieflower-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(14)
        .titleFontSize(24)
        .title("Project Timeline")
        .xAxisLabel("Time (cycles)")
        .yAxisLabel("Teams")
        .showGridLines(true)
        .categoryBackgroundOpacity([0.3, 0.05])
        .interactiveMode(true)
        .scrubberEnabled(true)
        .colorScheme(["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"])
        // Configure Google Fonts - Indie Flower (handwritten style)
        .titleFontFamily("Indie Flower, cursive")
        .axisFontFamily("Indie Flower, cursive") 
        .fontUrl("https://fonts.googleapis.com/css2?family=Indie+Flower&display=swap");
          
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Gantt Chart with Indie Flower (Google Fonts)"),
    Html.p("Indie Flower is a handwritten Google Font that works without CORS issues - a good alternative to Excalidraw fonts."),
    Html.div(id_=container_id, style="height: 400px;"),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

In [ ]:
# Gantt Chart with Google Fonts - Nunito (Clean Sans-Serif)
# A clean, professional alternative to Excalidraw's Nunito

container_id = f"d3-container-nunito-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(14)
        .titleFontSize(24)
        .title("Project Timeline")
        .xAxisLabel("Time (cycles)")
        .yAxisLabel("Teams")
        .showGridLines(true)
        .categoryBackgroundOpacity([0.3, 0.05])
        .interactiveMode(true)
        .scrubberEnabled(true)
        .colorScheme(["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"])
        // Configure Google Fonts - Nunito (clean sans-serif)
        .titleFontFamily("Nunito, sans-serif")
        .axisFontFamily("Nunito, sans-serif") 
        .fontUrl("https://fonts.googleapis.com/css2?family=Nunito&display=swap");
          
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Gantt Chart with Nunito (Google Fonts)"),
    Html.p("Nunito is a clean, readable sans-serif font - perfect for professional charts."),
    Html.div(id_=container_id, style="height: 400px;"),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

In [ ]:
# Gantt Chart with Google Fonts - Kalam (Best Comic Shanns Alternative)
# Kalam is a rounded, casual handwriting font - closest match to Comic Shanns style

container_id = f"d3-container-kalam-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(14)
        .titleFontSize(24)
        .title("Project Timeline")
        .xAxisLabel("Time (cycles)")
        .yAxisLabel("Teams")
        .showGridLines(true)
        .categoryBackgroundOpacity([0.3, 0.05])
        .interactiveMode(true)
        .scrubberEnabled(true)
        .colorScheme(["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"])
        // Configure Google Fonts - Kalam (best Comic Shanns alternative)
        .titleFontFamily("Kalam, cursive")
        .axisFontFamily("Kalam, cursive") 
        .fontUrl("https://fonts.googleapis.com/css2?family=Kalam&display=swap");
          
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Gantt Chart with Kalam (Google Fonts)"),
    Html.p("Kalam is a rounded, casual handwriting font - the closest Google Fonts alternative to Comic Shanns style."),
    Html.div(id_=container_id, style="height: 400px;"),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

In [ ]:
# Gantt Chart with Google Fonts - Architects Daughter
# Playful architectural handwriting style - another great Comic Shanns alternative

container_id = f"d3-container-architects-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart()
        .axisFontSize(14)
        .titleFontSize(24)
        .title("Project Timeline")
        .xAxisLabel("Time (cycles)")
        .yAxisLabel("Teams")
        .showGridLines(true)
        .categoryBackgroundOpacity([0.3, 0.05])
        .interactiveMode(true)
        .scrubberEnabled(true)
        .colorScheme(["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"])
        // Configure Google Fonts - Architects Daughter
        .titleFontFamily("Architects Daughter, cursive")
        .axisFontFamily("Architects Daughter, cursive") 
        .fontUrl("https://fonts.googleapis.com/css2?family=Architects+Daughter&display=swap");
          
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Gantt Chart with Architects Daughter (Google Fonts)"),
    Html.p("Architects Daughter has a playful, architectural handwriting feel - great for casual diagrams."),
    Html.div(id_=container_id, style="height: 400px;"),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

## Custom Font Support

The Gantt chart now supports custom fonts for titles and axis labels. This is particularly useful when you want the chart to match annotations you'll add manually (e.g., using Excalidraw).

### Font Configuration Options:

- **`.titleFontFamily(fontFamily)`**: Set the font family for the chart title
- **`.axisFontFamily(fontFamily)`**: Set the font family for axis labels and tick labels
- **`.fontUrl(url)`**: URL to load a custom font file (WOFF2) or CSS file (Google Fonts)

### Using Excalidraw Fonts:

#### ✅ **Virgil** (Publicly Accessible - RECOMMENDED)
The original Excalidraw handwritten font with permissive CORS settings:

```javascript
chart
    .titleFontFamily("Virgil, Segoe UI Emoji")
    .axisFontFamily("Virgil, Segoe UI Emoji")
    .fontUrl("https://virgil.excalidraw.com/Virgil.woff2");
```

#### ⚠️ **Other Excalidraw Fonts (CORS Restricted)**

Unfortunately, the newer Excalidraw fonts (**Excalifont**, **Comic Shanns**, **Nunito**) are hosted on `excalidraw.com` with CORS policies that only allow loading from the Excalidraw website itself. This means they **cannot be loaded** from Jupyter notebooks or other external origins.

### Google Fonts Alternatives:

Since most Excalidraw fonts are CORS-restricted, here are the **best Google Fonts alternatives**:

#### **For Comic Shanns → Use These:**

```javascript
// Kalam - Best match! Rounded, casual handwriting
chart
    .titleFontFamily("Kalam, cursive")
    .axisFontFamily("Kalam, cursive")
    .fontUrl("https://fonts.googleapis.com/css2?family=Kalam&display=swap");

// Architects Daughter - Playful, architectural handwriting
chart
    .titleFontFamily("Architects Daughter, cursive")
    .axisFontFamily("Architects Daughter, cursive")
    .fontUrl("https://fonts.googleapis.com/css2?family=Architects+Daughter&display=swap");

// Patrick Hand - Friendly, neat casual handwriting
chart
    .titleFontFamily("Patrick Hand, cursive")
    .axisFontFamily("Patrick Hand, cursive")
    .fontUrl("https://fonts.googleapis.com/css2?family=Patrick+Hand&display=swap");
```

#### **Other Handwritten Styles:**

```javascript
// Indie Flower - Casual handwritten annotations
chart
    .titleFontFamily("Indie Flower, cursive")
    .axisFontFamily("Indie Flower, cursive")
    .fontUrl("https://fonts.googleapis.com/css2?family=Indie+Flower&display=swap");

// Caveat - Natural handwritten feel
chart
    .titleFontFamily("Caveat, cursive")
    .axisFontFamily("Caveat, cursive")
    .fontUrl("https://fonts.googleapis.com/css2?family=Caveat&display=swap");

// Schoolbell - Childlike, playful (more informal)
chart
    .titleFontFamily("Schoolbell, cursive")
    .axisFontFamily("Schoolbell, cursive")
    .fontUrl("https://fonts.googleapis.com/css2?family=Schoolbell&display=swap");
```

#### **For Nunito (Clean Sans-Serif):**

```javascript
chart
    .titleFontFamily("Nunito, sans-serif")
    .axisFontFamily("Nunito, sans-serif")
    .fontUrl("https://fonts.googleapis.com/css2?family=Nunito&display=swap");
```

**How it works:** The chart automatically detects Google Fonts CSS URLs and loads them as `<link>` tags instead of trying to parse them as font files.

### Self-Host Excalidraw Fonts (Advanced):

If you absolutely need the exact Excalidraw fonts:

1. Install: `npm install @excalidraw/excalidraw`
2. Copy fonts from: `node_modules/@excalidraw/excalidraw/dist/prod/fonts/`
3. Host them on your own server with permissive CORS headers
4. Use the direct `.woff2` file URL with your server

### Font Loading Types:

The chart supports two font loading methods:

1. **CSS Files** (Google Fonts): Detected by `.css` extension or `fonts.googleapis.com` domain
   - Loaded as `<link rel="stylesheet">` tag
   - Example: `https://fonts.googleapis.com/css2?family=Kalam&display=swap`

2. **Direct Font Files** (.woff2): Any other URL
   - Loaded via `@font-face` CSS rule
   - Example: `https://virgil.excalidraw.com/Virgil.woff2`

### Font Application

The font is loaded dynamically and applied to:
- Chart title
- X-axis label
- Y-axis label  
- X-axis tick labels
- Y-axis category labels

### Summary:

- ✅ **Virgil**: Works perfectly, matches Excalidraw's original handwritten style
- ✅ **Kalam, Architects Daughter, Patrick Hand**: Best Comic Shanns alternatives
- ✅ **Indie Flower, Caveat**: Other handwritten styles
- ✅ **Nunito**: Clean sans-serif alternative
- ❌ **Excalifont, Comic Shanns**: CORS blocked from CDN
- ✅ **Google Fonts**: CSS URLs work automatically, no CORS issues